In [1]:
pip install requests beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests                       #importing the necessary packages
from bs4 import BeautifulSoup 
import re



def data_scrap(ticker):              # function definition
    
    PClose=''                         #creating the strings for the value sets needed
    Open=''
    Volume=''
    Cap=''
    scrapeddataset=[] 

    fin_url = 'https://finance.yahoo.com/quote/' + ticker + '?p=' + ticker   # yahoo url with ticker symbol
    fin_req = requests.get(fin_url)                                          # sending request for url

    fin_page = BeautifulSoup(fin_req.text, 'html.parser')                    # scrap html page using beatifulsoup

    fin_data = fin_page.find_all(class_="Bxz(bb) Bdbw(1px) Bdbs(s) Bdc($seperatorColor) H(36px)")  # filtering data of html page using class
    
    for val1 in fin_data:  
        scrapeddataset += [a.text for a in val1.find_all("span")] 

    for val2 in range(len(scrapeddataset)):               # accessing Open Price, Prev Close Price, Volume and Market Cap and storing them in variables

        if(scrapeddataset[val2] == "Previous Close"): 
            PClose = scrapeddataset[val2+1]

        if(scrapeddataset[val2] == "Open"): 
            Open = scrapeddataset[val2+1] 

        if(scrapeddataset[val2] == "Volume"): 
            Volume = scrapeddataset[val2+1]
            Volume = re.sub(r'[^\w\s]','',Volume)

        if(scrapeddataset[val2] == "Market Cap"):
            Cap = scrapeddataset[val2+1]    
    
    return(PClose,Open,Volume,Cap)


# main program starts here........


try:
                                                   
    f=open("stocks.csv",'w+')                      #creating file name and intiating write mode in file
                                                   
    comp_req = requests.get('https://money.cnn.com/data/hotstocks/')  #using the requests lib get the url for scrapping
                                                   
    comp_page = BeautifulSoup(comp_req.text, 'html.parser')  #using beautifulsoup library , scrapping the page using html.parser
                                                   
    comp_data = comp_page.find_all(class_="wsod_dataTable wsod_dataTableBigAlt")  #using the find_all function filtering out all the data present with class specified 

    values =[]
    title = ['Most Actives', 'Gainers', 'Losers'] 
    
    for data in comp_data: 
        
        #extracting all the needed data and values from the scrapped webpage using find_all func through for loop
                                                   
        [tmp.extract() for tmp in data.find_all(class_ = "wsod_aRight")]  
        abbr = [a.text for a in data.find_all(class_= "wsod_symbol")] 
        name = [a.text for a in data.find_all("span")] 
        values += [dict(zip(abbr, name))]                        # arranging the data set using dict func

       
    finaldata = dict(zip(title, values))         #this list contains company ticker symbols and name sorted using dict func 

    space=' '
    count=0

    print("Which stock are you interested in:")

    while count<=2:
        
        fd=finaldata[title[count]]
        title_val=(title[count])
        print('\n',title_val,'\n') 
        
        f.write(title_val)      # storing titles: Most Actives, Gainers and Losers in file.
        f.write('\n\n')
        
        for i in fd:

            print(i,fd[i])   # printing ticker symbol and company name

                    #print(i)
            PClose,Open,Volume,Cap = data_scrap(i)  # call function to scrap data based on ticker symbol
                        
            outputs=i,',',fd[i],',',PClose,',',Open,',',Volume,',',Cap,'\n'
            add_space=space.join(outputs)

            f.write(add_space)
                        

        count=count+1
    f.close()          # closing file here

    while True:
        temp_val = False

        symbol_inp = input("\nEnter your Interested Company :  ")    #asking user to input the company name 
        symbol_inp = symbol_inp.upper()
        
        file_open=open('stocks.csv')                                 # open file
        words=file_open.read().strip().split('\n')
        for a in words:
            line=a.split(',')
            
            for b in line:
                symbol=b.strip()

                    #print(ask)
                if (symbol==symbol_inp):
                    PClose,Open,Volume,Cap = data_scrap(symbol_inp)             #call scrap function again to get data from website
                    
                    print('\n')
                    print("The data for ",line[0].strip(),line[1].strip(),"is the following:\n")
                    print(line[0].strip(),line[1].strip())
                    print('Prev close:',  PClose)
                    print('Open: ', Open)
                    print('Volume: ', Volume)
                    print('Market Cap: ', Cap)
                    temp_val = True
                    break
                    
            if (symbol==symbol_inp):
                break

        if temp_val:
            break
        else:
            print ("\nYou did not enter correct ticker symbol from the list.")
except:
    print("Please re run the code!")

Which stock are you interested in:

 Most Actives 

GE General Electric Co
BAC Bank of America Corp
F Ford Motor Co
CCL Carnival Corp
DAL Delta Air Lines Inc
BA Boeing Co
OXY Occidental Petroleum Corp
MRO Marathon Oil Corp
WFC Wells Fargo & Co
C Citigroup Inc

 Gainers 

HFC HollyFrontier Corp
MPC Marathon Petroleum Corp
CFG Citizens Financial Group Inc
CMA Comerica Inc
VLO Valero Energy Corp
HES Hess Corp
ADS Alliance Data Systems Corp
BA Boeing Co
EOG EOG Resources Inc
DVN Devon Energy Corp

 Losers 

UNH UnitedHealth Group Inc
CNC Centene Corp
CLX Clorox Co
CPB Campbell Soup Co
ANTM Anthem Inc
NEM Newmont Corporation
HUM Humana Inc
KEYS Keysight Technologies Inc
CI Cigna Corp
WMT Walmart Inc

Enter your Interested Company :  CLX


The data for  CLX Clorox Co is the following:

CLX Clorox Co
Prev close: 196.49
Open:  194.12
Volume:  2361659
Market Cap:  24.207B
